In [32]:
import requests

body = {"jobs": [
            {"id": 1, "service": 300, "amount": [1], "location":[19.98935, 48.701], "skills":[1], "time_windows":[[32400, 36000]]}, 
            {"id": 2, "service": 300, "amount": [1], "location":[2.03655, 48.61128], "skills":[1]}, 
            {"id": 3, "service": 300, "amount": [1], "location":[2.39719, 49.07611], "skills":[2]}, 
            {"id": 4, "service": 300, "amount": [1], "location":[2.41808, 49.22619], "skills":[2]}, 
            {"id": 5, "service": 300, "amount": [1], "location":[2.28325, 48.5958], "skills":[14]}, 
            {"id": 6, "service": 300, "amount": [1], "location":[2.89357, 48.90736], "skills":[14]}], 
        "vehicles": [{"id": 1, "profile": "driving-car", 
            "start": [22.35044, 48.71764], "end":[24.35044, 48.71764], 
            "capacity":[4], "skills":[1, 14], "time_window":[28800, 43200]}]}

headers = {
    'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
    'Authorization': '5b3ce3597851110001cf62489ebebb576d5f468d914755e3f36e9829',
    'Content-Type': 'application/json; charset=utf-8'
}
call = requests.post(
    'https://api.openrouteservice.org/optimization', json=body, headers=headers)

print(call.status_code, call.reason)
print(call.text)

200 OK
{"code":0,"summary":{"cost":0,"unassigned":6,"delivery":[0],"amount":[0],"pickup":[0],"service":0,"duration":0,"waiting_time":0,"computing_times":{"loading":314,"solving":0}},"unassigned":[{"id":6,"location":[2.89357,48.90736]},{"id":5,"location":[2.28325,48.5958]},{"id":4,"location":[2.41808,49.22619]},{"id":3,"location":[2.39719,49.07611]},{"id":1,"location":[19.98935,48.701]},{"id":2,"location":[2.03655,48.61128]}],"routes":[]}



In [20]:
import json
# dump call.text to sample.json file with formatting
with open('sample.json', 'w') as f:
    json.dump(json.loads(call.text), f, indent=4)
    


In [22]:
# plot the route on a map
import folium
from folium import plugins
import pandas as pd
import numpy as np
import requests

# read the json file
with open('sample.json') as f:
    data = json.load(f)

# get the coordinates of the route
coordinates = []
for i in range(len(data['routes'][0]['steps'])):
    coordinates.append(data['routes'][0]['steps'][i]['location'])

# get the coordinates of the vehicles
vehicles = []
for i in range(len(data['routes'][0]['steps'])):
    if data['routes'][0]['steps'][i]['type'] == 'start':
        vehicles.append(data['routes'][0]['steps'][i]['location'])

# get the coordinates of the jobs
jobs = []
for i in range(len(data['routes'][0]['steps'])):
    if data['routes'][0]['steps'][i]['type'] == 'job':
        jobs.append(data['routes'][0]['steps'][i]['location'])

# create a map
m = folium.Map(location=[48.71764, 2.35044], zoom_start=10)

# add the route to the map
folium.PolyLine(coordinates, color="red", weight=2.5, opacity=1).add_to(m)

# add the vehicles to the map
for i in range(len(vehicles)):
    folium.Marker(location=vehicles[i], icon=folium.Icon(color='blue', icon='car', prefix='fa')).add_to(m)

# add the jobs to the map
for i in range(len(jobs)):
    folium.Marker(location=jobs[i], icon=folium.Icon(color='green', icon='home', prefix='fa')).add_to(m)

# save the map
m.save('map.html')

# display the map
m
